Your Google Drive should look like this:

My Drive/
└── capstone_project/
    └── CROHME_2023/
        └── train/
            ├── train_001.inkml
            ├── train_002.inkml
            └── ...
Now, you can create a new Colab notebook and follow these steps.

In [ ]:
import zipfile

zip_path = '/content/drive/MyDrive/capstone/TC11_CROHME23.zip'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('dataset')  # Change the destination folder as needed


Cell 1: Setup and Mount Google Drive

This cell will install the necessary libraries and connect your notebook to your Google Drive, giving you access to the CROHME data.

In [ ]:
# Install required libraries
!pip install lxml gtts

# Import necessary libraries and mount Google Drive
from google.colab import drive
import os
import csv
from lxml import etree
from gtts import gTTS
import time

print("Mounting Google Drive...")
drive.mount('/content/drive')
print("Drive mounted successfully!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1
Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive mounted successfully!


Cell 2: Configuration - Set Your Project Paths

This is the most critical cell to modify. Point the paths to the folders you created in your Google Drive.

In [ ]:
# --- !! MODIFY THIS SECTION !! ---

# The root of your project folder in Google Drive
DRIVE_PROJECT_ROOT = "/content/drive/MyDrive/capstone_project"

# --- Don't modify below this line unless you changed the folder names ---

# Path to the root of your CROHME 2023 dataset within your Drive
CROHME_ROOT = os.path.join(DRIVE_PROJECT_ROOT, "CROHME_2023")

# List of all the training directories you want to include
INPUT_DIRS = [
    os.path.join(CROHME_ROOT, "train", "CROHME2023_train"),
    os.path.join(CROHME_ROOT, "train", "CROHME2019"),
    os.path.join(CROHME_ROOT, "train", "Artificial_data", "gen_LaTeX_data_CROHME_2023_corpus")
]
# --- Don't modify below this line ---

# Directory where the generated audio files will be saved (this will be created)
OUTPUT_AUDIO_DIR = os.path.join(CROHME_ROOT, "generated_speech")

# Path for the final CSV file that will link everything together
METADATA_CSV_PATH = os.path.join(CROHME_ROOT, "multimodal_metadata.csv")

# Verify that the input directories exist
all_dirs_found = True
for input_dir in INPUT_DIRS:
    if not os.path.isdir(input_dir):
        print(f"ERROR: Input directory not found at '{input_dir}'")
        all_dirs_found = False

if all_dirs_found:
    print(f"Project paths configured successfully.")
    print(f"Input data will be read from: {INPUT_DIRS}")
    print(f"Output audio will be saved to: {OUTPUT_AUDIO_DIR}")
else:
    print("Please check your folder names and paths in Google Drive.")

Project paths configured successfully.
Input data will be read from: ['/content/drive/MyDrive/capstone_project/CROHME_2023/train/CROHME2023_train', '/content/drive/MyDrive/capstone_project/CROHME_2023/train/CROHME2019', '/content/drive/MyDrive/capstone_project/CROHME_2023/train/Artificial_data/gen_LaTeX_data_CROHME_2023_corpus']
Output audio will be saved to: /content/drive/MyDrive/capstone_project/CROHME_2023/generated_speech


Cell 3: Helper Functions

This cell contains the core logic for parsing files, converting text, and generating audio. No modifications are needed here.

In [ ]:
def extract_latex_from_inkml(inkml_path):
    """Parses an .inkml file to extract the LaTeX ground truth."""
    try:
        # Create a parser that can recover from document errors
        parser = etree.XMLParser(recover=True)

        ns = {'inkml': 'http://www.w3.org/2003/InkML'}
        # Use the tolerant parser to read the file
        tree = etree.parse(inkml_path, parser)

        truth_element = tree.find(".//inkml:annotation[@type='truth']", namespaces=ns)
        # Some files might have the truth in a different annotation type
        if truth_element is None:
            truth_element = tree.find(".//inkml:annotation[@type='truth_LG']", namespaces=ns)

        return truth_element.text.strip() if truth_element is not None and truth_element.text is not None else None
    except Exception as e:
        print(f"  -> Error parsing {os.path.basename(inkml_path)}: {e}")
        return None

def latex_to_spoken_text(latex_string):
    """Converts a LaTeX string to a more natural spoken form (simplified)."""
    replacements = {
        "\\frac": " fraction ", "\\sqrt": " square root of ", "\\sum": " sum from ",
        "\\int": " integral from ", "\\in": " in ", "\\lim": " limit as ",
        "\\log": " log ", "\\sin": " sin ", "\\cos": " cos ", "\\tan": " tan ",
        "\\alpha": " alpha ", "\\beta": " beta ", "\\gamma": " gamma ", "\\pi": " pi ",
        "^": " to the power of ", "_": " sub ", "{": " start ", "}": " end ",
        "=": " equals ", "+": " plus ", "-": " minus ", "(": "", ")": "",
    }
    text = latex_string.replace("\\left", "").replace("\\right", "")
    for old, new in replacements.items():
        text = text.replace(old, new)
    text = text.replace("\\", "")
    return ' '.join(text.split())

def generate_audio(text, output_path):
    """Generates an audio file from text using gTTS."""
    try:
        tts = gTTS(text=text, lang='en', slow=False)
        tts.save(output_path)
        return True
    except Exception as e:
        print(f"  -> gTTS Error for text '{text[:30]}...': {e}")
        return False

Cell 4: Main Processing Script - Run the Generation

This is the final cell that executes the entire process. It includes logic to resume if your Colab session disconnects, which is very important for a long-running task.

⚠️ Warning: Generating 12,000+ audio files can take several hours. It's highly recommended to test this on a small subset first.

In [ ]:
# Create the output directory if it doesn't exist
os.makedirs(OUTPUT_AUDIO_DIR, exist_ok=True)

# --- RESUME LOGIC ---
# Check if a metadata file already exists to find out what's been processed.
processed_files = set()
if os.path.exists(METADATA_CSV_PATH):
    print("Found existing metadata file. Will resume from where we left off.")
    with open(METADATA_CSV_PATH, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        next(reader) # Skip header
        for row in reader:
            # Assumes inkml_file path is in the first column
            processed_files.add(os.path.basename(row[0]))
    print(f"Found {len(processed_files)} previously processed files.")

# --- FILE GATHERING ---
all_inkml_files = []
for input_dir in INPUT_DIRS:
    all_inkml_files.extend([os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith(".inkml")])

files_to_process = [f for f in all_inkml_files if os.path.basename(f) not in processed_files]

# --- !! FOR TESTING: UNCOMMENT THE LINE BELOW TO RUN ON A SMALL SUBSET !! ---
# files_to_process = files_to_process[:20]

print(f"\nTotal files found across all directories: {len(all_inkml_files)}")
print(f"Files remaining to process: {len(files_to_process)}")

# --- MAIN LOOP ---
header = ['inkml_file_path', 'audio_file_path', 'latex_ground_truth', 'spoken_text']

# Open the CSV in append mode ('a') to add new rows
with open(METADATA_CSV_PATH, 'a', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    # Write header only if the file is new (empty)
    if not processed_files:
        writer.writerow(header)

    for i, inkml_path in enumerate(files_to_process):
        filename = os.path.basename(inkml_path)
        print(f"Processing ({i+1}/{len(files_to_process)}): {filename}")

        latex_truth = extract_latex_from_inkml(inkml_path)
        if not latex_truth:
            print(f"  -> Skipping, no LaTeX found.")
            continue

        spoken_text = latex_to_spoken_text(latex_truth)

        audio_filename = filename.replace(".inkml", ".mp3")
        audio_path = os.path.join(OUTPUT_AUDIO_DIR, audio_filename)

        success = generate_audio(spoken_text, audio_path)
        if not success:
            print(f"  -> Skipping, audio generation failed.")
            continue

        # Write the new row to the CSV
        writer.writerow([inkml_path, audio_path, latex_truth, spoken_text])

        # Be polite to the TTS service, especially from a Google IP
        time.sleep(1)

print("\n--- SCRIPT FINISHED ---")
print(f"All processed data is saved in your Google Drive at: {CROHME_ROOT}")

Found existing metadata file. Will resume from where we left off.
Found 11354 previously processed files.

Total files found across all directories: 12389
Files remaining to process: 1036
Processing (1/1036): MfrDB1094.inkml
Processing (2/1036): 200922-947-220.inkml
Processing (3/1036): formulaire018-equation056.inkml
Processing (4/1036): MfrDB1581.inkml
Processing (5/1036): 36_em_31.inkml
Processing (6/1036): 115_em_139.inkml
Processing (7/1036): formulaire019-equation028.inkml
Processing (8/1036): MfrDB0892.inkml
Processing (9/1036): formulaire022-equation007.inkml
Processing (10/1036): 200926-1617-60.inkml
Processing (11/1036): formulaire010-equation019.inkml
Processing (12/1036): formulaire043-equation067.inkml
Processing (13/1036): TrainData2_17_sub_73.inkml
Processing (14/1036): MfrDB0400.inkml
Processing (15/1036): 2009213-137-13.inkml
Processing (16/1036): KME2G3_2_sub_95.inkml
Processing (17/1036): 200926-1617-163.inkml
Processing (18/1036): 200924-1331-181.inkml
Processing (1

## Your Final Output: The metadata.csv

After the script runs, you will have the metadata.csv file in your Google Drive. It will look something like this:

inkml_file_path,audio_file_path,latex_ground_truth and spoken_text                          



This file is "ready to use" because it contains everything your model's data loader will need:

The path to the handwriting data.

The path to the corresponding speech data.

The ground truth label (LaTeX string) that the model needs to learn to predict.

## How You'll Use It in Your Model Training

Your next step will be to write a custom Dataset class (in PyTorch or a tf.data.Dataset in TensorFlow). This class will:

Read the metadata.csv file at the beginning.

For each sample (__getitem__), it will use the paths from one row of the CSV to:

Load the handwriting file.

Load the audio file.

Get the LaTeX label.

Pre-process this data and feed it directly to your model for training.

Because the script creates this clean, organized manifest file, you can immediately start working on the modeling part of your project without any more data management. 🚀